# Parallel Chains

Similarly to how sequential chains connect chains in series, parallel chains connect chains in parallel. This is a useful abstraction when you want to independently process the same input with multiple different chains, possibly asynchronously.

In the following examples, we wlll show:
- how we can use `ParallelChain` to take a list of chains and apply each independently to the same input.
- how we can nest `ParallelChain`s inside `ParallelChain`s

In [1]:
import pprint
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ParallelChain

In [2]:
"""
In Jupyter notebooks, the default event loop policy is set to asyncio.get_event_loop_policy().ipython_kernel, 
which is different from the default event loop policy used by the Python interpreter. 
This can cause issues if we're running code that assumes the default event loop policy is being used.

To resolve this issue, we set the event loop policy in the Jupyter notebook to match the 
one used by the Python interpreter.
"""
import nest_asyncio

# Set the event loop policy to the default used by the Python interpreter
nest_asyncio.apply()

## Simple Example

In [3]:
llm = OpenAI(temperature=0.9)

input_variables=['product']

prompt_1 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good mission statement for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

prompt_3 = PromptTemplate(
    input_variables=input_variables,
    template="What is a good slogan for a company that makes {product}?",
)
chain_3 = LLMChain(llm=llm, prompt=prompt_3)

prompt_4 = PromptTemplate(
    input_variables=input_variables,
    template="What are some core values for a company that makes {product}?",
)
chain_4 = LLMChain(llm=llm, prompt=prompt_4)

By setting the `concurrent` flag to `True`, we can run the child chains concurrently.

In [4]:
parallel_chain = ParallelChain(
    input_variables=input_variables,
    chains={
        'name': chain_1, 
        'mission': chain_2, 
        'slogan': chain_3,
        'values': chain_4
    },
    verbose=True,
    concurrent=True
)
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Concurrent executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="name" finished after 0.71 seconds.
Child chain for key="slogan" finished after 0.84 seconds.
Child chain for key="mission" finished after 1.77 seconds.
Child chain for key="values" finished after 4.09 seconds.

> Finished chain.
{'mission/text': '\n'
                 '\n'
                 'Our mission at [Company Name] is to provide customers with '
                 'distinctive and fashionable socks that match their '
                 'individual style and personality. We strive to create '
                 'quality products that bring joy and self-expression to '
                 'everyday life.',
 'name/text': '\n\nArgyle Appeal.',
 'product': 'colorful socks',
 'slogan/text': '\n\n"Step into Color with Our Socks!"',
 'values/text': '\n'
                '\n'
                

Setting the `concurrent` flag to `False` would run the child chains serially.

In [5]:
parallel_chain.concurrent=False
s = time.perf_counter()
output = parallel_chain("colorful socks")
pprint.pprint(output)
print('\033[1m' + f"Serial executed in {time.perf_counter()-s:0.2f} seconds." + '\033[0m')



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 3.79 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 1.84 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 0.65 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 3.55 seconds.

> Finished chain.
{'mission/text': '\n'
                 '\n'
                 'Our mission at [Company Name] is to provide stylish, '
                 'comfortable and quality socks that bring a smile to our '
                 "customers' faces. We strive to always be colorful, creative "
                 'and innovative in our designs and provide outstanding '
                 'customer service.',
 'name/text': '\n\nVivid Sockery.',
 'product': 'colorful socks',
 'slogan/text': '\n\n"Step into a World of Color with Our Socks!"',
 'values/text': '\n'
                '\n

## Nesting `ParallelChain`s
It is possible to nest `ParallelChain`s inside one another. Continuing from the previous example, we nest a concurrent `ParallelChain` inside the previous serial `ParallelChain`.

In [6]:
prompt_5_1 = PromptTemplate(
    input_variables=input_variables,
    template="Which gift would go well with {product}?",
)
chain_5_1 = LLMChain(llm=llm, prompt=prompt_5_1)

prompt_5_2 = PromptTemplate(
    input_variables=input_variables,
    template="What gift would not go well with {product}?",
)
chain_5_2 = LLMChain(llm=llm, prompt=prompt_5_2)

chain_5 = ParallelChain(
    input_variables=input_variables,
    chains={'good_gift': chain_5_1, 'bad_gift': chain_5_2},
    verbose=True,
    concurrent=True
)

parallel_chain.chains.update({'gift': chain_5})

output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="name" finished after 0.50 seconds.
Child chain for key="mission" started.
Child chain for key="mission" finished after 1.77 seconds.
Child chain for key="slogan" started.
Child chain for key="slogan" finished after 0.61 seconds.
Child chain for key="values" started.
Child chain for key="values" finished after 4.32 seconds.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="bad_gift" finished after 0.47 seconds.
Child chain for key="good_gift" finished after 0.97 seconds.

> Finished chain.
Child chain for key="gift" finished after 0.98 seconds.

> Finished chain.
{'gift/bad_gift/text': '\n\nA book.',
 'gift/good_gift/text': '\n'
                        '\n'
                        'A pack of colorful shoe laces or a pack of colorful '
                        'shoela

Now we make the outer `ParallelChain` execute concurrently again. The following shows an example execution of a concurrent `ParallelChain` nested inside another concurrent `ParallelChain`.

In [7]:
parallel_chain.concurrent=True
output = parallel_chain("colorful socks")
pprint.pprint(output)



> Entering new ParallelChain chain...
Child chain for key="name" started.
Child chain for key="mission" started.
Child chain for key="slogan" started.
Child chain for key="values" started.
Child chain for key="gift" started.


> Entering new ParallelChain chain...
Child chain for key="good_gift" started.
Child chain for key="bad_gift" started.
Child chain for key="slogan" finished after 0.70 seconds.Child chain for key="bad_gift" finished after 0.70 seconds.

Child chain for key="name" finished after 0.71 seconds.
Child chain for key="good_gift" finished after 1.06 seconds.

> Finished chain.
Child chain for key="gift" finished after 1.06 seconds.
Child chain for key="mission" finished after 1.83 seconds.
Child chain for key="values" finished after 4.89 seconds.

> Finished chain.
{'gift/bad_gift/text': '\n\nA book.',
 'gift/good_gift/text': '\n'
                        '\n'
                        'A pair of brightly-colored slippers or sandals would '
                        'go we